In [1]:
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
import glob
import tqdm
import sys
import os
import csv
import cv2
import random
from PIL import Image

np.random.bit_generator = np.random._bit_generator

In [2]:
img_dir = "/Users/Work/Documents/Conversion/data/dataset/"
aug_dir = "/Users/Work/Documents/Conversion/data/datasetAug/"

## Define number of augmented images to generate
aug_num = 10

## Bounding box coordinates: [[(image1 box1), (image1 box2)], [(image2 box1), (image2 box2)]]
coor = []
img_list = []
img_concat = np.array([])

In [10]:
def tilting():
    dataIn = open("/Users/Work/Documents/Conversion/data/csvAug.txt", "r")
    dataOut = open("/Users/Work/Documents/Conversion/data/cs.txt", "w")

    with dataIn as dataInRead:
        readValues = csv.reader(dataInRead, delimiter=',')

        count = 0

        for row in readValues:
            if count == 0:
                dataOut.write("\"{}\", {}, {}, {}, {}, {}\n".format(row[0], row[1], row[2], row[3], row[4], row[5]))
            else:
                for i in range(0, 5):
                    ia.seed(1)
                    image = cv2.imread("/Users/Work/Documents/Conversion/data/dataset/" + row[0])

                    h, w, c = image.shape

                    bbs = ia.BoundingBoxesOnImage(
                     [ia.BoundingBox(x1=round(float(row[1])*w), y1= round(float(row[2])*h), x2= round(float(row[3])*w), y2= round(float(row[4])*h))],
                     shape=image.shape)

                    seq = iaa.Sequential([
                            iaa.Affine(
                                rotate=(-25,25),
                            ),
                            iaa.Fliplr(p=np.random.uniform(0, 1)),
                            iaa.Flipud(p=np.random.uniform(0, 1))
                            ])

                    seq_det = seq.to_deterministic()

                    image_aug = seq_det.augment_images([image])[0]
                    bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]
                    cv2.imwrite("/Users/Work/Documents/Conversion/data/datasetAug/{}{}".format((i+4), row[0]), image_aug)

                    xMinNorm = (float(bbs_aug.bounding_boxes[0].x1) / w)
                    yMinNorm = (float(bbs_aug.bounding_boxes[0].y1) / h)
                    xMaxNorm = (float(bbs_aug.bounding_boxes[0].x2) / w)
                    yMaxNorm = (float(bbs_aug.bounding_boxes[0].y2) / h)

                    dataOut.write("\"{}{}\", {}, {}, {}, {}, {}\n".format((i+4), row[0], xMinNorm, yMinNorm, xMaxNorm, yMaxNorm, row[5]))
                
            count += 1

    dataIn.close()
    dataOut.close()

In [11]:
tilting()

In [ ]:
bbs_aug.bounding_boxes[0]

In [ ]:
## Concat images
for i in tqdm.tqdm(range(len(img_list))):
    img_array = np.asarray(Image.open(img_list[i]))
    img_concat = np.append(img_concat, np.expand_dims(img_array, axis=0))

img_concat = np.reshape(img_concat, ((len(img_list), img_array.shape[0], img_array.shape[1], 3)))
img_concat = img_concat.astype(np.uint8)

  0%|▎                                                                          | 18/3737 [20:47<279:31:14, 270.58s/it]

In [ ]:
## Concat bounding box
bounding_boxes = []

for each in tqdm.tqdm(range(len(coor))):
    container = []

    for each_box in coor[each]:
        container.append(ia.BoundingBox(x1=int(each_box[0]), y1=int(each_box[1]), x2=int(each_box[2]), y2=int(each_box[3]),))

    bounding_boxes.append(container)

In [ ]:
## Augmenting images

aug_box_list = [] ## Format of output of augmented box coordinates same as input

for j in tqdm.tqdm(range(int(aug_num/len(img_list)))):
    seq = iaa.Sequential([
        iaa.Multiply(mul=np.random.uniform(0.5, 1.5)),
        iaa.Fliplr(p=np.random.uniform(0, 1)),
        iaa.Flipud(p=np.random.uniform(0, 1))
        ])

    img_aug, bbs_aug = seq(images=img_concat, bounding_boxes=bounding_boxes)

    for i, each_img_bbs in enumerate(bbs_aug):
        aug_container = []
        img_aug_each = Image.fromarray(img_aug[i])
        img_aug_each.save(os.path.join(aug_dir, (img_list[i].split("/")[-1].split(".")[0] + "_" + str(j+1) + ".png")))

        for each_bbs in each_img_bbs:
            aug_container.append((each_bbs.x1, each_bbs.y1, each_bbs.x2, each_bbs.y2))

        aug_box_list.append(aug_container)